# Project 2

Griffith and Becca

# Q: How does not getting an appropriate of sleep effect the spread of sickness?

An inappropriate amount of sleep - either too little or too much - can weaken the immune system. This can make it easier for individuals to become infected by a sickness. 

# This model aims to determine how having different percentages of the population with different amounts of sleep effects the spread of a sickness accross that population.

# To Do
* Fix make_system to work such that infected and susceptible add up to 1000, right now they add up to 2000.
* Survey with Olin students, how much sleep do they get?
* Implement number of Olin students into total enrollment (335). Make sure to prevent fractional students!

In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

This model relies on a lot of stocks (11 to be specific). There are susceptible and infected stocks for all sleep values (As the contact rate is different for each).

In [5]:
def make_system(sleepFive, sleepSix, sleepSeven, sleepEight, sleepNine, infectedInitial, beta5, beta6, beta7, beta8, beta9, beta, gamma):
    """Make a system object for the SIR model.
    
    sleepFive: Percentage of the population with five or less hours of sleep
    sleepSix: Percentage of the population with six hours of sleep
    sleepSeven: Percentage of the population with seven hours of sleep
    sleepEight: Percentage of the population with eight hours of sleep
    sleepNine: Percentage of the population with nine or more hours of sleep
    infectedInitial: Percentage of the population that starts out infected
    
    beta5: contact rate multiplier in for five hours of sleep
    beta6: contact rate multiplier for six hours of sleep
    beta7: contact rate multiplier for seven hours of sleep
    beta8: contact rate multiplier for eight hours of sleep
    beta9: contact rate multiplier for nine hours of sleep
    beta: contact rate in days
    gamma: recovery rate in days
    
    returns: System object
    """
    TotalPopulation = 1000
    
    S5 = sleepFive*TotalPopulation
    S6 = sleepSix*TotalPopulation
    S7 = sleepSeven*TotalPopulation
    S8 = sleepEight*TotalPopulation
    S9 = sleepNine*TotalPopulation 
    
    I = TotalPopulation * infectedInitial
    
    init = State(S5=S5, S6=S6, S7=S7, S8=S8, S9=S9, 
                 I=I, R=0)
    t0 = 0
    t_end = 7 * 14
    

    return System(init=init, t0=t0, t_end=t_end,
                  beta5=beta5, beta6=beta6, beta7=beta7, beta8=beta8, beta9=beta9,
                  beta=beta, gamma=gamma)

There are two plot functions. One plots the total Susceptible, Infected, and Recovered (SIR) and prints out a statement with the percentages of each. The other plot function plots all of the stocks (All of the various sleep values for susceptible and infected).

# FIX PLOT FUNCTION and total infected (If needed)

In [6]:
def plot_results(results):
    """Plot the results of a SIR model.
    
    results: Dataframe with the results of the model
    """
    plot(results.S5, '--', label='Susceptible')
    plot(I, '-', label='Infected')
    plot(R, ':', label='Recovered')
    decorate(xlabel='Time (days)',
             ylabel='Fraction of population')

In [7]:
def calc_total_infected(results):
    """Fraction of population infected during the simulation.
    
    results: DataFrame with columns S, I, R
    
    returns: fraction of population
    """
    return get_first_value(results.S) - get_last_value(results.S)

Uses the make_system function to make a system with the below fractions of people who get five, six, seven, eight, and nine hours of sleep. Also specify the portion of the population that starts out infected.

In [8]:
sleepFive = 0.1
sleepSix = 0.1
sleepSeven = 0.1
sleepNine = 0.1
sleepEight = 1 - sleepFive - sleepSix - sleepSeven - sleepNine
initialInfected = .05

#Numbers made up currently
beta5 = 1.3 #contact rate multiplier in for five hours of sleep
beta6 = 1.2 #contact rate multiplier for six hours of sleep
beta7 = 1.1 #contact rate multiplier for seven hours of sleep
beta8 = 1.0 #contact rate multiplier for eight hours of sleep
beta9 = 1.1 #contact rate multiplier for nine hours of sleep
beta = 0.333
gamma = 0.25
system = make_system(sleepFive, sleepSix, sleepSeven, sleepEight, sleepNine, initialInfected, beta5, beta6, beta7, beta8, beta9, beta, gamma)

,values
init,S5 100.0 S6 100.0 S7 100.0 S8 600....
t0,0
t_end,98
beta5,1.3
beta6,1.2
beta7,1.1
beta8,1
beta9,1.1
beta,0.333
gamma,0.25


In [17]:
def slope_func(state, t, system):
    """Update the SIR model.
    
    state: State (s5, s6, s7, s8, s9, i5, i6, i7, i8, i9, r)
    t: time
    system: System object
    
    returns: pair of derivatives
    """ 
    #s5, s6, s7, s8, s9, i5, i6, i7, i8, i9, r = state
    s5, s6, s7, s8, s9, i, r = state
    unpack(system)
    
    s = s5 + s6 + s7 + s8 + s9 # Total susceptible
    #i = i5 + i6 + i7 + i8 + i9 # Total infected
    
    ds5dt = -(beta*beta5) * i * (s5/s) #The change in the 5 hour of susceptible sleep group = infection rate * total infected * percent of susceptible that have 5 hours of sleep
    #di5dt = ds5dt - gamma * (i5/i) #Change in 5 hour of sleep infected group = change in susceptible group becoming infected - (recovery rate * percentage of infected that have 5 hours of sleep)
    
    ds6dt = -(beta*beta6) * i * (s6/s)
    #di6dt = ds6dt - gamma * (i6/i)
    
    ds7dt = -(beta*beta7) * i * (s7/s)
    #di7dt = ds7dt - gamma * (i7/i)
    
    ds8dt = -(beta*beta8) * i * (s8/s)
    #di8dt = ds8dt - gamma * (i8/i)
    
    ds9dt = -(beta*beta9) * i * (s9/s)
    #di9dt = ds9dt - gamma * (i9/i)
    
    TotalSchange = ds5dt + ds6dt + ds7dt + ds8dt + ds9dt
    didt = -TotalSchange - gamma * i
    drdt = gamma * i
   
    print('Check to make sure Total Infected Change = -(Total Susceptible Change) - (Total Recovered Change)')
    print('-(Total Susceptible Change) - (Total Recoverd Change) = ' , -TotalSchange - drdt)
    print('Total Infected Change = ' , didt)
    
    #return ds5dt, di5dt, ds6dt, di6dt, ds7dt, di7dt, ds8dt, di8dt, ds9dt, di9dt, drdt
    return ds5dt, ds6dt, ds7dt, ds8dt, ds9dt, didt, drdt

In [18]:
slope_func(system.init, system.t0, system)

Check to make sure Total Infected Change = -(Total Susceptible Change) - (Total Recovered Change)
-(Total Susceptible Change) - (Total Recoverd Change) =  5.315500000000004
Total Infected Change =  5.315500000000004


(-2.1645000000000003,
 -1.9979999999999998,
 -1.8315000000000003,
 -9.990000000000002,
 -1.8315000000000003,
 5.315500000000004,
 12.5)

In [19]:
system.init

,values
S5,100.0
S6,100.0
S7,100.0
S8,600.0
S9,100.0
I,50.0
R,0.0


In [ ]:
results.plot()

In [ ]:
results, details = run_ode_solver(system, slope_func, max_step = 2)
details

In [51]:
results.plot()

NameError: name 'results' is not defined